In [1]:
import os
import sys

sys.stderr = open(os.devnull, "w")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.seasonal import STL, MSTL
from sktime.forecasting.tbats import TBATS
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import median_abs_deviation
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Dropout

In [2]:
sales = pd.read_csv('pedidos.csv', sep=';')

In [3]:
sales

,created_date,account_id,sales_channel_id,price_total,status
0,2023-08-26 22:28:43.046 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,30.28,canceled
1,2023-08-26 22:28:43.047 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,73.38,canceled
2,2023-08-26 22:28:43.476 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,212.11,canceled
3,2023-08-26 22:28:44.657 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,24.72,canceled
4,2023-08-26 22:28:45.692 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,149.50,canceled
...,...,...,...,...,...
18279636,2024-06-11 22:46:26.615 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,3,39.99,NaN
18279637,2023-08-26 22:28:37.609 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,32.39,canceled
18279638,2023-08-26 22:28:42.596 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,3,49.50,canceled
18279639,2023-08-26 22:28:43.045 -0300,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,6,28.34,canceled


In [4]:
sales.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18279641 entries, 0 to 18279640
Data columns (total 5 columns):
 #   Column            Non-Null Count     Dtype  
---  ------            --------------     -----  
 0   created_date      18279641 non-null  object 
 1   account_id        18279641 non-null  object 
 2   sales_channel_id  18279641 non-null  int64  
 3   price_total       18279641 non-null  float64
 4   status            15035149 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 697.3+ MB


In [5]:
sales.describe()

,sales_channel_id,price_total
count,1.827964e+07,1.827964e+07
mean,4.265544e+00,1.035530e+03
std,6.297053e+00,3.736278e+03
min,1.000000e+00,-4.216000e+01
25%,1.000000e+00,1.378100e+02
50%,1.000000e+00,3.601600e+02
75%,6.000000e+00,1.310950e+03
max,4.300000e+01,6.598500e+06


In [6]:
df = sales[['account_id', 'sales_channel_id']].value_counts()

pd.DataFrame(df).reset_index()

,account_id,sales_channel_id,count
0,5f5ed10c-7792-4080-970c-b11398e3f58f,1,3004186
1,9d39683a-47b3-4a18-942d-f1d741a47e8c,1,2521471
2,05d9cc3a-decc-4fa2-b4e8-b44eef40e3b6,3,1895786
3,5f5ed10c-7792-4080-970c-b11398e3f58f,9,1057418
4,74abffac-6542-4eff-8eae-cc85463a5d02,1,981137
...,...,...,...
248,5f5ed10c-7792-4080-970c-b11398e3f58f,38,3
249,60ca8452-5b14-481f-a637-8756f527aee8,4,2
250,f9129295-bb08-4330-b60c-9f0beadda521,36,1
251,457bce7b-9300-4c10-9a97-070b3c0d081d,5,1


In [7]:
sales['created_date'].duplicated().sum()

np.int64(364414)

In [ ]:
# TEST_SIZE = 24
LOOKBACK = 3
SEASONAL_PERIODS = (24, 24*7)
END_DATE = pd.to_datetime(pd.to_datetime(sales['created_date'].max()).strftime("%Y-%m-%d %H:00:00"))
START_DATE = END_DATE - timedelta(hours=END_DATE.hour)
TEST_SIZE = int((END_DATE - START_DATE).seconds / 60**2 + 1)

In [ ]:
OOT_DATE = START_DATE + timedelta(hours=23)

In [ ]:
OOT_PERIODS = int((OOT_DATE - END_DATE).seconds / 60**2 + 1)

In [ ]:
oot_dates = pd.DatetimeIndex([END_DATE+timedelta(hours=h) for h in range(1, OOT_PERIODS)], freq='h')
df_oot = pd.DataFrame(index=oot_dates)

In [ ]:
account_id = 'f9129295-bb08-4330-b60c-9f0beadda521'
sales_channel_id = 1

# cond = (sales['account_id'] == account_id) & (sales['sales_channel_id'] == sales_channel_id)
# df_client = sales[cond].drop(['account_id', 'sales_channel_id'], axis=1)
# df_client['created_date'] = pd.to_datetime(df_client['created_date'], format='%Y-%m-%d %H:%M:%S.%f %z')
# df_client = df_client.sort_values('created_date').reset_index(drop=True)

cond = (sales['account_id'] == account_id) & (sales['sales_channel_id'] == sales_channel_id)
df_client = sales[cond].drop(['account_id', 'sales_channel_id'], axis=1)
df_client['created_date'] = pd.to_datetime(df_client['created_date'], format='%Y-%m-%d %H:%M:%S.%f %z')
df_client = df_client.sort_values('created_date').reset_index(drop=True)
df_client['created_date'] = df_client['created_date'].dt.strftime("%Y-%m-%d %H:00:00").reset_index(drop=True)

df_client_mod = df_client.groupby('created_date').agg(price_total_agg=('price_total', 'sum'), n_orders=('created_date', 'count'))
df_client_mod.index = pd.to_datetime(df_client_mod.index, format='%Y-%m-%d %H:00:00')

end_date = pd.to_datetime(END_DATE, format='%Y-%m-%d %H:%M:%S')
start_date = end_date - relativedelta(months=LOOKBACK)
date_index = pd.Series(pd.date_range(start=start_date, end=end_date, freq='h', name='created_date'))
df_client_mod = pd.merge(date_index, df_client_mod, how='left', on='created_date').set_index('created_date')

In [ ]:
# df_client

In [ ]:
# df_client.info(show_counts=True)

In [ ]:
# df_client.describe()

In [ ]:
# df_client['created_date'] = df_client['created_date'].dt.strftime("%Y-%m-%d %H").reset_index(drop=True)

In [ ]:
# df_client

In [ ]:
# df_client_mod = df_client.groupby('created_date').agg(price_total_agg=('price_total', 'sum'), n_orders=('created_date', 'count'))
# df_client_mod.index = pd.to_datetime(df_client_mod.index, format='%Y-%m-%d %H')

# df_client_mod

In [ ]:
# end_date = pd.to_datetime('2025-03-17 10')
# date_index = pd.Series(pd.date_range(start=end_date - relativedelta(months=3),
#                                      end=end_date,
#                                      freq='h',
#                                      name='created_date'))
# df_client_mod = pd.merge(date_index, df_client_mod, how='left', on='created_date').set_index('created_date')

In [ ]:
df_client_mod

# Price Total

## Time Series

In [ ]:
price_total = df_client_mod['price_total_agg']

In [ ]:
price_total

In [ ]:
price_total.isna().sum()

In [ ]:
price_total = price_total.fillna(0)

In [ ]:
price_total.isna().sum()

In [ ]:
lags = [price_total.shift(l).rename(f'lag_{l}') for l in range(1, 25)]
price_total_lag = pd.concat([price_total, pd.concat(lags, axis=1)], axis=1).dropna()

In [ ]:
# price_total.corr()['price_total_agg']

In [ ]:
# df = price_total[['price_total_agg', 'lag_1']]
df = price_total

In [ ]:
df_train = df[:-TEST_SIZE]
df_test = df[-TEST_SIZE:]

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(df_train, lw=0.8);

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(price_total.loc[end_date-timedelta(days=2):end_date], lw=0.8)
plt.xticks(rotation=90);

In [ ]:
res_stl = STL(df_train, period=24).fit()

In [ ]:
fig, ax = plt.subplots(4, 1, sharex=True, figsize=(12, 12))
fig.tight_layout()

res_stl.observed.plot(lw=0.8, ax=ax[0])
ax[0].set_ylabel("Observed")

res_stl.trend.plot(lw=0.8, ax=ax[1])
ax[1].set_ylabel("Trend")

res_stl.seasonal.plot(lw=0.8, ax=ax[2])
ax[2].set_ylabel("Seasonal")

res_stl.resid.plot(style=".", markersize=1, ax=ax[3])
ax[3].set_ylabel("Residual");

In [ ]:
trend = res_stl.trend

plt.figure(figsize=(12, 3))
sns.lineplot(trend.loc[end_date-timedelta(days=35):end_date], lw=0.8)
plt.xticks(rotation=90);

In [ ]:
res_mstl = MSTL(df_train, periods=(24, 24*7)).fit()

In [ ]:
fig, ax = plt.subplots(5, 1, sharex=True, figsize=(12, 12))
fig.tight_layout()

res_mstl.observed.plot(lw=0.8, ax=ax[0])
ax[0].set_ylabel("Observed")

res_mstl.trend.plot(lw=0.8, ax=ax[1])
ax[1].set_ylabel("Trend")

res_mstl.seasonal['seasonal_24'].plot(lw=0.8, ax=ax[2])
ax[2].set_ylabel("Seasonal 24h")

res_mstl.seasonal['seasonal_168'].plot(lw=0.8, ax=ax[3])
ax[3].set_ylabel("Seasonal 7d")

res_mstl.resid.plot(style=".", markersize=1, ax=ax[4])
ax[4].set_ylabel("Residual");

In [ ]:
model = TBATS(sp=[24, 168], show_warnings=False, n_jobs=-1)
model.fit(df_train)

In [ ]:
forecast = model.predict(fh=df.index)
forecast_int = model.predict_interval(fh=df.index, coverage=0.95)

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(df_train, lw=0.8, label='Train')
sns.lineplot(forecast[:-TEST_SIZE], lw=0.8, label='Estimation');

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(df_train.iloc[-6*TEST_SIZE:], lw=0.8, label='Train')
sns.lineplot(forecast[-5*TEST_SIZE:-TEST_SIZE], lw=0.8, label='Estimation')
sns.lineplot(df_test, lw=0.8, label='Test')
sns.lineplot(forecast[-TEST_SIZE:], lw=0.8, label='Forecast')
sns.lineplot(forecast_int.iloc[-TEST_SIZE:, 0], c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(forecast_int.iloc[-TEST_SIZE:, 1], c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(2*forecast_int.iloc[-TEST_SIZE:, 0]-forecast[-TEST_SIZE:], c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
sns.lineplot(2*forecast_int.iloc[-TEST_SIZE:, 1]-forecast[-TEST_SIZE:], c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
plt.fill_between(x=forecast_int.index[-TEST_SIZE:],
                 y1=forecast_int.iloc[-TEST_SIZE:, 0],
                 y2=forecast_int.iloc[-TEST_SIZE:, 1],
                 color='violet',
                 alpha=0.5)
plt.fill_between(x=forecast_int.index[-TEST_SIZE:],
                 y1=2*forecast_int.iloc[-TEST_SIZE:, 0]-forecast[-TEST_SIZE:],
                 y2=2*forecast_int.iloc[-TEST_SIZE:, 1]-forecast[-TEST_SIZE:],
                 color='violet',
                 alpha=0.3)
plt.legend();

In [ ]:
train_pred = forecast[:-TEST_SIZE]

train_mae = mean_absolute_error(df_train, train_pred)
train_rmse = root_mean_squared_error(df_train, train_pred)

print(f"RMSE: {train_rmse:.3f}")
print(f"MAE:  {train_mae:.3f}")

In [ ]:
test_pred = forecast[-TEST_SIZE:] 

mae = mean_absolute_error(df_test, test_pred)
rmse = root_mean_squared_error(df_test, test_pred)

print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")

## Machine Learning

In [ ]:
test_split_date = end_date - timedelta(days=1)
df_train = price_total.loc[price_total.index <= test_split_date].copy()
df_test = price_total.loc[price_total.index > test_split_date].copy()

X_train, y_train = df_train.drop('price_total_agg', axis=1), df_train['price_total_agg']
X_test, y_test = df_test.drop('price_total_agg', axis=1), df_test['price_total_agg']

In [ ]:
tscv = TimeSeriesSplit()

In [ ]:
lr = LinearRegression()
cv_results = cross_validate(lr,
                            X=X_train,
                            y=y_train,
                            scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error'],
                            cv=tscv,
                            n_jobs=-1,
                            return_train_score=True)
cv_results = pd.DataFrame(cv_results)
print('train RMSE:', -cv_results['train_neg_root_mean_squared_error'].mean())
print('valid RMSE:', -cv_results['test_neg_root_mean_squared_error'].mean())
print('train MAE:', -cv_results['train_neg_mean_absolute_error'].mean())
print('valid MAE:', -cv_results['test_neg_mean_absolute_error'].mean())

In [ ]:
gb = GradientBoostingRegressor(max_depth=10, random_state=42, n_iter_no_change=3)

In [ ]:
valid_split_date = end_date - timedelta(days=2)

df_valid = df_train.loc[(df_train.index > valid_split_date) & (df_train.index <= test_split_date)].copy()
X_valid, y_valid = df_valid.drop('price_total_agg', axis=1), df_valid['price_total_agg']

df_train_mod = df_train.loc[df_train.index <= valid_split_date].copy()
X_train_mod, y_train_mod = df_train_mod.drop('price_total_agg', axis=1), df_train_mod['price_total_agg']

In [ ]:
# gb.fit(X_train_mod, y_train_mod)
# estimation = pd.Series(gb.predict(X_train_mod), index=X_train_mod.index)
# forecast = pd.Series(gb.predict(X_valid), index=X_valid.index)

# print('train RMSE:', root_mean_squared_error(y_train_mod, estimation))
# print('valid RMSE:', root_mean_squared_error(y_valid, forecast))
# print('train MAE:', mean_absolute_error(y_train_mod, estimation))
# print('valid MAE:', mean_absolute_error(y_valid, forecast))

In [ ]:
# model = gb

In [ ]:
LOWER_ALPHA = 0.025
MED_ALPHA = 0.5
UPPER_ALPHA = 0.975

lower_gb = GradientBoostingRegressor(loss="quantile", max_depth=10, random_state=42, n_iter_no_change=3, alpha=LOWER_ALPHA)
lower_gb.fit(X_train, y_train)

gb = GradientBoostingRegressor(loss="quantile", max_depth=10, random_state=42, n_iter_no_change=3, alpha=0.5)
gb.fit(X_train, y_train)

upper_gb = GradientBoostingRegressor(loss="quantile", max_depth=10, random_state=42, n_iter_no_change=3, alpha=UPPER_ALPHA)
upper_gb.fit(X_train, y_train);

# model.fit(X_train, y_train);

In [ ]:
estimation = pd.Series(model.predict(X_train), index=X_train.index)

lower = pd.Series(lower_gb.predict(X_test), index=X_test.index)
upper = pd.Series(upper_gb.predict(X_test), index=X_test.index)

y_pred_test = gb.predict(X_test)
forecast = pd.Series(y_pred_test, index=X_test.index)

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train, lw=0.8, label='Train')
sns.lineplot(estimation, lw=0.8, label='Estimation');

In [ ]:
np.random.seed(42)

In [ ]:
# def bootstrap_models(X, y, n_bootstraps=100):
#     models = []
#     for k in range(n_bootstraps):
#         idx = np.random.choice(len(X), size=len(X), replace=True)
#         X_boot, y_boot = X.iloc[idx], y.iloc[idx]
#         model = GradientBoostingRegressor(max_depth=10, random_state=42, n_iter_no_change=3)
#         model.fit(X_boot, y_boot)
#         models.append(model)
#     return models

In [ ]:
# ensemble = bootstrap_models(X_train, y_train)

In [ ]:
# predictions = np.column_stack([model.predict(X_test) for model in ensemble])

# lower = pd.Series(np.percentile(predictions, 2.5, axis=1), index=X_test.index)
# upper = pd.Series(np.percentile(predictions, 97.5, axis=1), index=X_test.index)

# forecast = pd.Series(np.percentile(predictions, 50, axis=1), index=X_test.index)

In [ ]:
print('test RMSE:', root_mean_squared_error(y_test, forecast))
print('test MAE:', mean_absolute_error(y_test, forecast))

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train[-3*TEST_SIZE:], lw=0.8, label='Train')
sns.lineplot(estimation[-3*TEST_SIZE:], lw=0.8, label='Estimation')
sns.lineplot(y_test, lw=0.8, label='Test')
sns.lineplot(forecast, lw=0.8, label='Forecast')
sns.lineplot(lower, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(upper, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(2*lower-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
sns.lineplot(2*upper-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
plt.fill_between(x=forecast.index, y1=lower, y2=upper, color='violet', alpha=0.5)
plt.fill_between(x=forecast.index, y1=2*lower-forecast, y2=2*upper-forecast, color='violet', alpha=0.3)
plt.legend();

## Deep Learning

In [ ]:
test_split_date = end_date - timedelta(days=1)
df_train = price_total_lag.loc[price_total_lag.index <= test_split_date].copy()
df_test = price_total_lag.loc[price_total_lag.index > test_split_date].copy()

X_train, y_train = df_train.drop('price_total_agg', axis=1), df_train['price_total_agg']
X_test, y_test = df_test.drop('price_total_agg', axis=1), df_test['price_total_agg']

In [ ]:
tf.random.set_seed(42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled = np.reshape(X_train_scaled, (X_train.shape[0], 1, X_train.shape[1]))
X_test_scaled = np.reshape(X_test_scaled, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
def bootstrap_models(X, y, n_bootstraps=10):
    models = []
    i = 1
    for _ in range(n_bootstraps):
        print('#bootstrap:', i)
        idx = np.random.choice(len(X), size=len(X), replace=True)
        X_boot, y_boot = X.iloc[idx], y.iloc[idx]
        scaler = StandardScaler()
        X_boot_scaled = scaler.fit_transform(X_boot)
        X_boot_scaled = np.reshape(X_boot_scaled, (X_boot.shape[0], 1, X_boot.shape[1]))
        early_stopping = EarlyStopping(patience=3)
        model = Sequential()
        model.add(Input(shape=(1, 24)))
        model.add(LSTM(128, activation='relu', return_sequences=True))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.compile(loss='mean_absolute_error', optimizer='adam')
        model.fit(X_boot_scaled, y_boot, epochs=500, batch_size=32, validation_split=0.1, shuffle=False, callbacks=early_stopping, verbose=2)
        models.append(model)
        i += 1
    return models

In [ ]:
ensemble = bootstrap_models(X_train, y_train)
predictions = np.column_stack([model.predict(X_test_scaled) for model in ensemble])

In [ ]:
lower = pd.Series(np.percentile(predictions.squeeze(), 15, axis=1), index=X_test.index)
forecast = pd.Series(np.percentile(predictions.squeeze(), 50, axis=1), index=X_test.index)
upper = pd.Series(np.percentile(predictions.squeeze(), 85, axis=1), index=X_test.index)

In [ ]:
# model = Sequential()
# model.add(Input(shape=(1, 24)))
# model.add(LSTM(128, activation='relu', return_sequences=True))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.compile(loss='mean_absolute_error', optimizer='adam')
# model.fit(X_train_scaled, y_train, epochs=100, batch_size=1, verbose=2)

In [ ]:
# estimation = pd.Series(model.predict(X_train_scaled).squeeze(), index=X_train.index)
# forecast = pd.Series(model.predict(X_test_scaled).squeeze(), index=X_test.index)

predictions = np.column_stack([model.predict(X_train_scaled) for model in ensemble])
estimation = pd.Series(np.percentile(predictions.squeeze(), 50, axis=1), index=X_train.index)

In [ ]:
print('train RMSE:', root_mean_squared_error(y_train, estimation))
print('test RMSE:', root_mean_squared_error(y_test, forecast))
print('train MAE:', mean_absolute_error(y_train, estimation))
print('test MAE:', mean_absolute_error(y_test, forecast))

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train, lw=0.8, label='Train')
sns.lineplot(estimation, lw=0.8, label='Estimation');

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train[-5*TEST_SIZE:], lw=0.8, label='Train')
sns.lineplot(estimation[-5*TEST_SIZE:], lw=0.8, label='Estimation')
sns.lineplot(y_test, lw=0.8, label='Test')
sns.lineplot(forecast, lw=0.8, label='Forecast')
sns.lineplot(lower, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(upper, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(2*lower-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
sns.lineplot(2*upper-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
plt.fill_between(x=forecast.index, y1=lower, y2=upper, color='violet', alpha=0.5)
plt.fill_between(x=forecast.index, y1=2*lower-forecast, y2=2*upper-forecast, color='violet', alpha=0.3)
plt.legend();

# Number of Orders

## Time Series

In [ ]:
orders = df_client_mod['n_orders']

In [ ]:
orders

In [ ]:
orders.isna().sum()

In [ ]:
orders = orders.fillna(0)

In [ ]:
orders.isna().sum()

In [ ]:
# lags = [orders.shift(l).rename(f'lag_{l}') for l in range(1, 25)]
# orders_lag = pd.concat([orders, pd.concat(lags, axis=1)], axis=1).dropna()

In [ ]:
# orders_lag.corr()['n_orders']

In [ ]:
# df = orders_lag[['n_orders', 'lag_1']]
df = orders

In [ ]:
df_train = df.iloc[:-TEST_SIZE]
df_test = df.iloc[-TEST_SIZE:]

In [ ]:
plt.figure(figsize=(12, 3))
# sns.lineplot(df_train['n_orders'], lw=0.8)
sns.lineplot(df_train, lw=0.8);

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(orders.loc[end_date-timedelta(days=2):end_date], lw=0.8)
plt.xticks(rotation=90);

In [ ]:
# res_stl = STL(df_train['n_orders'], period=24).fit()
res_stl = STL(df_train, period=24).fit()

In [ ]:
fig, ax = plt.subplots(4, 1, sharex=True, figsize=(12, 12))
fig.tight_layout()

res_stl.observed.plot(lw=0.8, ax=ax[0])
ax[0].set_ylabel("Observed")

res_stl.trend.plot(lw=0.8, ax=ax[1])
ax[1].set_ylabel("Trend")

res_stl.seasonal.plot(lw=0.8, ax=ax[2])
ax[2].set_ylabel("Seasonal")

res_stl.resid.plot(style=".", markersize=1, ax=ax[3])
ax[3].set_ylabel("Residual");

In [ ]:
trend = res_stl.trend

plt.figure(figsize=(12, 3))
sns.lineplot(trend.loc[end_date-timedelta(days=35):end_date], lw=0.8)
plt.xticks(rotation=90);

In [ ]:
# res_mstl = MSTL(df_train['n_orders'], periods=(24, 24*7)).fit()
res_mstl = MSTL(df_train, periods=(24, 24*7)).fit()

In [ ]:
fig, ax = plt.subplots(5, 1, sharex=True, figsize=(12, 12))
fig.tight_layout()

res_mstl.observed.plot(lw=0.8, ax=ax[0])
ax[0].set_ylabel("Observed")

res_mstl.trend.plot(lw=0.8, ax=ax[1])
ax[1].set_ylabel("Trend")

res_mstl.seasonal['seasonal_24'].plot(lw=0.8, ax=ax[2])
ax[2].set_ylabel("Seasonal 24h")

res_mstl.seasonal['seasonal_168'].plot(lw=0.8, ax=ax[3])
ax[3].set_ylabel("Seasonal 7d")

res_mstl.resid.plot(style=".", markersize=0.8, lw=0.8, ax=ax[4])
ax[4].set_ylabel("Residual");

In [ ]:
model = TBATS(sp=[24, 168], show_warnings=False, n_jobs=-1)
# model.fit(df_train['n_orders'], X=df_train['lag_1'])
model.fit(df_train)

In [ ]:
# forecast = model.predict(fh=df.index, X=df['lag_1'])
# forecast_int = model.predict_interval(fh=df.index, X=df['lag_1'], coverage=0.99)
forecast = model.predict(fh=df.index)
forecast_int = model.predict_interval(fh=df.index, coverage=0.45)

In [ ]:
# plt.figure(figsize=(12, 3))
# sns.lineplot(df_train['n_orders'], lw=0.8, label='Train')
# sns.lineplot(forecast[:-TEST_SIZE], lw=0.8, label='Estimation');
plt.figure(figsize=(12, 3))
sns.lineplot(df_train, lw=0.8, label='Train')
sns.lineplot(forecast[:-TEST_SIZE], lw=0.8, label='Estimation');

In [ ]:
plt.figure(figsize=(12, 3))
# sns.lineplot(df_train.iloc[-3*TEST_SIZE:, 0], lw=0.8, label='Train')
sns.lineplot(df_train.iloc[-6*TEST_SIZE:], lw=0.8, label='Train')
sns.lineplot(forecast[-5*TEST_SIZE:-TEST_SIZE], lw=0.8, label='Estimation')
# sns.lineplot(df_test['n_orders'], lw=0.8, label='Test')
sns.lineplot(df_test, lw=0.8, label='Test')
sns.lineplot(forecast[-TEST_SIZE:], lw=0.8, label='Forecast')
sns.lineplot(forecast_int.iloc[-TEST_SIZE:, 0], c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(forecast_int.iloc[-TEST_SIZE:, 1], c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(2*forecast_int.iloc[-TEST_SIZE:, 0]-forecast[-TEST_SIZE:], c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
sns.lineplot(2*forecast_int.iloc[-TEST_SIZE:, 1]-forecast[-TEST_SIZE:], c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
plt.fill_between(x=forecast_int.index[-TEST_SIZE:],
                 y1=forecast_int.iloc[-TEST_SIZE:, 0],
                 y2=forecast_int.iloc[-TEST_SIZE:, 1],
                 color='violet',
                 alpha=0.5)
plt.fill_between(x=forecast_int.index[-TEST_SIZE:],
                 y1=2*forecast_int.iloc[-TEST_SIZE:, 0]-forecast[-TEST_SIZE:],
                 y2=2*forecast_int.iloc[-TEST_SIZE:, 1]-forecast[-TEST_SIZE:],
                 color='violet',
                 alpha=0.3)
plt.legend();

In [ ]:
train_pred = forecast[:-TEST_SIZE]

train_mae = mean_absolute_error(df_train, train_pred)
train_rmse = root_mean_squared_error(df_train, train_pred)

print(f"RMSE: {train_rmse:.3f}")
print(f"MAE:  {train_mae:.3f}")

In [ ]:
test_pred = forecast[-TEST_SIZE:] 

mae = mean_absolute_error(df_test, test_pred)
rmse = root_mean_squared_error(df_test, test_pred)

print(f"RMSE: {rmse:.3f}")
print(f"MAE:  {mae:.3f}")

## Machine Learning

In [ ]:
test_split_date = end_date - timedelta(days=1)
df_train = orders_lag.loc[orders_lag.index <= test_split_date].copy()
df_test = orders_lag.loc[orders_lag.index > test_split_date].copy()

X_train, y_train = df_train.drop('n_orders', axis=1), df_train['n_orders']
X_test, y_test = df_test.drop('n_orders', axis=1), df_test['n_orders']

In [ ]:
tscv = TimeSeriesSplit()

In [ ]:
lr = LinearRegression()
cv_results = cross_validate(lr,
                            X=X_train,
                            y=y_train,
                            scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_error'],
                            cv=tscv,
                            n_jobs=-1,
                            return_train_score=True)
cv_results = pd.DataFrame(cv_results)
print('train RMSE:', -cv_results['train_neg_root_mean_squared_error'].mean())
print('valid RMSE:', -cv_results['test_neg_root_mean_squared_error'].mean())
print('train MAE:', -cv_results['train_neg_mean_absolute_error'].mean())
print('valid MAE:', -cv_results['test_neg_mean_absolute_error'].mean())

In [ ]:
gb = GradientBoostingRegressor(max_depth=10, random_state=42, n_iter_no_change=3)

In [ ]:
valid_split_date = end_date - timedelta(days=2)

df_valid = df_train.loc[(df_train.index > valid_split_date) & (df_train.index <= test_split_date)].copy()
X_valid, y_valid = df_valid.drop('n_orders', axis=1), df_valid['n_orders']

df_train_mod = df_train.loc[df_train.index <= valid_split_date].copy()
X_train_mod, y_train_mod = df_train_mod.drop('n_orders', axis=1), df_train_mod['n_orders']

In [ ]:
gb.fit(X_train_mod, y_train_mod)
estimation = pd.Series(gb.predict(X_train_mod), index=X_train_mod.index)
forecast = pd.Series(gb.predict(X_valid), index=X_valid.index)

print('train RMSE:', root_mean_squared_error(y_train_mod, estimation))
print('valid RMSE:', root_mean_squared_error(y_valid, forecast))
print('train MAE:', mean_absolute_error(y_train_mod, estimation))
print('valid MAE:', mean_absolute_error(y_valid, forecast))

In [ ]:
model = gb

In [ ]:
LOWER_ALPHA = 0.025
UPPER_ALPHA = 0.975

lower_gb = GradientBoostingRegressor(loss="quantile", max_depth=10, random_state=42, n_iter_no_change=3, alpha=LOWER_ALPHA)
lower_gb.fit(X_train, y_train)

upper_gb = GradientBoostingRegressor(loss="quantile", max_depth=10, random_state=42, n_iter_no_change=3, alpha=UPPER_ALPHA)
upper_gb.fit(X_train, y_train)

model.fit(X_train, y_train);

In [ ]:
estimation = pd.Series(model.predict(X_train), index=X_train.index)

lower = pd.Series(lower_gb.predict(X_test), index=X_test.index)
upper = pd.Series(upper_gb.predict(X_test), index=X_test.index)

y_pred_test = model.predict(X_test)
forecast = pd.Series(y_pred_test, index=X_test.index)

In [ ]:
print('test RMSE:', root_mean_squared_error(y_test, y_pred_test))
print('test MAE:', mean_absolute_error(y_test, y_pred_test))

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train, lw=0.8, label='Train')
sns.lineplot(estimation, lw=0.8, label='Estimation');

In [ ]:
plt.figure(figsize=(12, 3))
sns.lineplot(y_train[-3*TEST_SIZE:], lw=0.8, label='Train')
sns.lineplot(estimation[-3*TEST_SIZE:], lw=0.8, label='Estimation')
sns.lineplot(y_test, lw=0.8, label='Test')
sns.lineplot(forecast, lw=0.8, label='Forecast')
sns.lineplot(lower, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(upper, c='darkviolet', linestyle='--', lw=0.8)
sns.lineplot(2*lower-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
sns.lineplot(2*upper-forecast, c='darkviolet', alpha=0.3, linestyle='--', lw=0.8)
plt.fill_between(x=forecast.index, y1=lower, y2=upper, color='violet', alpha=0.5)
plt.fill_between(x=forecast.index, y1=2*lower-forecast, y2=2*upper-forecast, color='violet', alpha=0.3)
plt.legend();